In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [5]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [6]:
df.shape

(4026, 7)

In [7]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [9]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [10]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [11]:
y=normalized_df["Period (P)"]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

# ngboost

In [1]:
from ngboost import NGBRegressor

In [15]:
from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplg

In [20]:
def bayesopt_objective(n_estimators,learning_rate):
    model= NGBRegressor(n_estimators=int(n_estimators)
              ,learning_rate=learning_rate
              )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [21]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
    
    opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
   
    params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
  
    print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [22]:
def bayes_opt_validation(params_best):
    model=NGBRegressor(n_estimators=int(params_best['n_estimators'])
            ,learning_rate=params_best['learning_rate']
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [23]:
# Start time
start_time = time.time()

param_grid_simple={'n_estimators':(1000,5000)
                   ,'learning_rate':( 0.001,0.999)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best 
validation_score=bayes_opt_validation(params_best)
validation_score 
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.01005   | 0.6719    | 2.952e+03 |
| 2         | 0.01018   | 0.8248    | 1.126e+03 |
| 3         | 0.00935   | 0.8074    | 3.262e+03 |
| 4         | 0.01128   | 0.298     | 1.187e+03 |
| 5         | 0.01044   | 0.9896    | 1.027e+03 |
| 6         | 0.008945  | 0.7693    | 3.987e+03 |
| 7         | 0.009574  | 0.3777    | 2.977e+03 |
| 8         | 0.009083  | 0.9281    | 2.582e+03 |
| 9         | 0.009047  | 0.973     | 3.098e+03 |
| 10        | 0.009308  | 0.09443   | 4.253e+03 |
| 11        | 0.009466  | 0.2123    | 3.217e+03 |
| 12        | 0.009624  | 0.2927    | 4.265e+03 |
| 13        | 0.01002   | 0.8274    | 1.886e+03 |
| 14        | 0.0114    | 0.6445    | 1.381e+03 |
| 15        | 0.01061   | 0.4118    | 1.387e+03 |
| 16        | 0.00978   | 0.1447    | 1.849e+03 |
| 17        | 0.01026   | 0.4767    | 1.31e+03  |
| 18        | 0.01093   | 0.2356    | 1.026e+03 |


| 100       | 0.008826  | 0.4598    | 4.197e+03 |
| 101       | 0.01026   | 0.683     | 2.079e+03 |
| 102       | 0.01137   | 0.04437   | 1.644e+03 |
| 103       | 0.009758  | 0.1807    | 1.85e+03  |
| 104       | 0.009414  | 0.926     | 1.571e+03 |
| 105       | 0.01103   | 0.37      | 1.388e+03 |
| 106       | 0.009743  | 0.6869    | 3.861e+03 |
| 107       | 0.01174   | 0.7009    | 1.071e+03 |
| 108       | 0.01038   | 0.2141    | 1.761e+03 |
| 109       | 0.008774  | 0.8768    | 4.999e+03 |
| 110       | 0.008785  | 0.3478    | 4.285e+03 |
| 111       | 0.009053  | 0.8613    | 4.164e+03 |
| 112       | 0.01001   | 0.4942    | 1.388e+03 |
| 113       | 0.01067   | 0.7838    | 1.071e+03 |
| 114       | 0.009147  | 0.2597    | 4.795e+03 |
| 115       | 0.0105    | 0.7475    | 1.071e+03 |
| 116       | 0.01113   | 0.07374   | 1.017e+03 |
| 117       | 0.01208   | 0.05528   | 1.017e+03 |
| 118       | 0.01646   | 0.004727  | 1.644e+03 |
| 119       | 0.008752  | 0.4341    | 4.772e+03 |


In [24]:
from ngboost import NGBRegressor
params_best['n_estimators'] = int(params_best['n_estimators'])
nbr_reg= NGBRegressor(**params_best )
nbr_reg.fit(X_train, y_train)

[iter 0] loss=-0.0899 val_loss=0.0000 scale=1.0000 norm=0.4985
[iter 100] loss=-3.1627 val_loss=0.0000 scale=1.0000 norm=0.4446
[iter 200] loss=-3.5664 val_loss=0.0000 scale=0.5000 norm=0.2173
[iter 300] loss=-3.7803 val_loss=0.0000 scale=0.0039 norm=0.0017
[iter 400] loss=-3.8126 val_loss=0.0000 scale=1.0000 norm=0.4201
[iter 500] loss=-3.8830 val_loss=0.0000 scale=0.2500 norm=0.1035
[iter 600] loss=-3.9435 val_loss=0.0000 scale=0.0078 norm=0.0032
[iter 700] loss=-3.9980 val_loss=0.0000 scale=0.0078 norm=0.0032
[iter 800] loss=-4.0559 val_loss=0.0000 scale=0.0078 norm=0.0032
[iter 900] loss=-4.0923 val_loss=0.0000 scale=0.5000 norm=0.2046
[iter 1000] loss=-4.1161 val_loss=0.0000 scale=1.0000 norm=0.4064


NGBRegressor(learning_rate=0.8854, n_estimators=1017,
             random_state=RandomState(MT19937) at 0x23220E7D240)

In [25]:
y_pred= nbr_reg.predict(X_test)
y_pred1=nbr_reg.predict(X_val)
y_pred2=nbr_reg.predict(X_train)

In [26]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.005439384593051375
mean_squared_error: 5.6569356100853564e-05
rmse: 0.007521260273441783
r2 score: 0.9988654090187775


In [27]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.004014162036098418
mean_squared_error: 3.1328460140959686e-05
rmse: 0.005597183232748387
r2 score: 0.9993595365220106


In [28]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.005668775500325886
mean_squared_error: 6.50179537181907e-05
rmse: 0.008063371113758233
r2 score: 0.9987308781667686
